# SQLite Database Prototype

This notebook designs and prototypes a SQLite database for the SKU generation system, with the intent to migrate to PostgreSQL on a Synology NAS in the future.

## Design Principles
- Use SQLAlchemy ORM for database-agnostic code
- Design schema to support future PostgreSQL migration
- Maintain compatibility with existing Numbers-based workflow during transition

## Phase 1: Schema Design

### Current Numbers File Structure (Reference)

From `catalog.numbers`:
- **Product_Catalog sheet**: Main product models with metadata
- **Feature sheets**: Colors, Sizes, Designs, Materials, etc.
- Each product has multiple features that create permutations

### Proposed Database Schema

Below we'll design each table structure before implementation.

### Table 1: Categories

**Purpose**: Store product categories and subcategories

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| category | VARCHAR(100) | NOT NULL | Main category (e.g., "Apparel") |
| subcategory | VARCHAR(100) | | Subcategory (e.g., "T-Shirts") |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on (category, subcategory) for lookup performance

**Notes**: 
- Add any additional fields you need here

### Additional Tables to Consider

**What else do you need to track?**

Ideas:
- **Suppliers**: Vendor information
- **Images**: Product/SKU images (URLs or paths)
- **Pricing History**: Track price changes over time
- **Inventory Transactions**: Log inventory changes
- **Sales Channels**: Where products are sold (website, store, etc.)
- **Product Relationships**: Bundles, variants, related products
- **Custom Fields**: Flexible metadata storage

**Tell me what additional columns or tables you want to add!**

### Table 7: SKU Features

**Purpose**: Link each SKU to its specific feature combination

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| sku_id | INTEGER | FOREIGN KEY → skus.id | The SKU |
| feature_id | INTEGER | FOREIGN KEY → features.id | Feature in this SKU |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |

**Indexes**: 
- Index on sku_id
- Index on feature_id

**Constraints**:
- UNIQUE(sku_id, feature_id) - no duplicate features per SKU

**Notes**: 
- This allows querying all SKUs with a specific feature (e.g., "all Black products")

### Table 6: SKUs

**Purpose**: Store generated SKUs with all their feature combinations

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| sku | VARCHAR(100) | NOT NULL, UNIQUE | Generated SKU code |
| product_id | INTEGER | FOREIGN KEY → products.id | Base product |
| batch_name | VARCHAR(200) | | Batch identifier (for tracking) |
| price | DECIMAL(10,2) | | SKU-specific price (if different from base) |
| inventory_qty | INTEGER | DEFAULT 0 | Current inventory count |
| status | VARCHAR(20) | DEFAULT 'active' | active/inactive/discontinued |
| generated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | When SKU was generated |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on sku (already implied by UNIQUE)
- Index on product_id
- Index on batch_name
- Index on status

**Notes**: 
- What other SKU-level fields do you need? (weight, dimensions, UPC, etc.?)

### Table 5: Product Features

**Purpose**: Link products to their available features (which colors/sizes apply to which products)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| product_id | INTEGER | FOREIGN KEY → products.id | Product this applies to |
| feature_id | INTEGER | FOREIGN KEY → features.id | Available feature |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |

**Indexes**: 
- Index on product_id
- Index on feature_id

**Constraints**:
- UNIQUE(product_id, feature_id) - no duplicate feature assignments

**Notes**: 
- This junction table defines which features are valid for each product

### Table 4: Features

**Purpose**: Store individual feature values (specific colors, sizes, etc.)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| feature_type_id | INTEGER | FOREIGN KEY → feature_types.id | Type of feature |
| code | VARCHAR(20) | NOT NULL | Short code for SKU (e.g., "BLK", "XL") |
| name | VARCHAR(100) | NOT NULL | Display name (e.g., "Black", "Extra Large") |
| description | TEXT | | Additional details |
| sort_order | INTEGER | | Order within feature type |
| is_active | BOOLEAN | DEFAULT TRUE | Whether feature is currently available |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on (feature_type_id, code)
- Index on is_active

**Constraints**:
- UNIQUE(feature_type_id, code) - no duplicate codes within same feature type

**Notes**: 
- Any additional feature attributes you want to track?

### Table 3: Feature Types

**Purpose**: Define types of features (Color, Size, Design, etc.)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| feature_type | VARCHAR(50) | NOT NULL, UNIQUE | e.g., "Color", "Size", "Design" |
| display_order | INTEGER | | Order for display purposes |
| is_required | BOOLEAN | DEFAULT FALSE | Whether feature is required for SKU |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |

**Notes**: 
- This allows us to add new feature types without schema changes

### Table 2: Products

**Purpose**: Store base product models (before permutations)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| product_name | VARCHAR(200) | NOT NULL, UNIQUE | Product model name |
| category_id | INTEGER | FOREIGN KEY → categories.id | Link to category |
| description | TEXT | | Product description |
| base_price | DECIMAL(10,2) | | Base price (optional) |
| status | VARCHAR(20) | DEFAULT 'active' | active/inactive/discontinued |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on product_name
- Index on category_id
- Index on status

**Notes**: 
- What other product-level fields do you need?

### Draft Schema in Mermaid

**See the rendered ER diagram in `docs/schema-diagram.md`**

Open that file and press `Cmd+Shift+V` to view the Mermaid diagram with all table relationships.

## Phase 2: Data Migration

## Phase 3: SKU Generation Logic

## Phase 4: SKU Generation Logic

## Phase 5: PostgreSQL Preparation